In [1]:
from llama_index.core import ListIndex, SimpleDirectoryReader, VectorStoreIndex, Settings
from dotenv import load_dotenv
from llama_index.llms.ollama import  Ollama
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.ollama import OllamaEmbedding
import chromadb

In [ ]:
Settings.llm = Ollama(model="llama3.1:8b-instruct-fp16", request_timeout=360.0, temperature=0.2)
load_dotenv()

False

# List Index

In [40]:
documents = SimpleDirectoryReader("./data/ravens_web_official_news_10_7_10_14").load_data()
index = ListIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("What's new with Baltimore Ravens?")
print(response)

**Rewrite**

The Baltimore Ravens' offense is showing signs of improvement, particularly with Zay Flowers leading the charge. He had an impressive game against the Washington Commanders, setting multiple records for receiving yards in the first half and helping the team secure a win. Flowers has been heating up over the past two weeks, and his chemistry with Lamar Jackson and other receivers is evident. The Ravens' offense is confident in their abilities, and they're working hard to put it all together on a consistent basis. With players like Flowers making big plays, the team is optimistic about their chances of success moving forward.


# Tree Index

In [37]:
from llama_index.core import TreeIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('./data/ravens_web_official_news_10_7_10_14').load_data()
tree_index = TreeIndex.from_documents(documents)
query_engine = tree_index.as_query_engine()
response = query_engine.query("What's new with Baltimore Ravens?")
print(response)

The Baltimore Ravens are riding a four-game winning streak, with their offense looking like a juggernaut. They have a balanced attack that is powered by running back Derrick Henry, who has been historically good this season. He has become the first player since Pro Football Hall of Famer LaDainian Tomlinson in 2005 to score a rushing touchdown in each of his first six games of a season. The Ravens are also just the second team in the Super Bowl era with at least 150 rushing yards and a rushing touchdown in each of its first six games of a season.


# Keyword Table Index

In [38]:
from llama_index.core import KeywordTableIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('./data/ravens_web_official_news_10_7_10_14').load_data()
keyword_index = KeywordTableIndex.from_documents(documents)
query_engine = keyword_index.as_query_engine()
response = query_engine.query("What's new with Baltimore Ravens?")
print(response)

**Rewrite**: The Ravens' defense has been trending upward after a physical performance against the Commanders, with several key players making bone-jarring hits and swarming to the football. This was a big game for Baltimore's defense after giving up five touchdown passes in Week 5, but they were able to raise their game by eliminating glaring mistakes and limiting Washington's longest play from scrimmage to just 28 yards. The addition of veteran coach Dean Pees has likely contributed to this improvement, as the team looks to build on its strong rush defense and shore up its secondary.


# Vector Store Index

In [ ]:
# Create Chroma client
chroma_client = chromadb.EphemeralClient()

# Define collection name
collection_name = "ravens"

# Check if the collection already exists
existing_collections = chroma_client.list_collections()

if collection_name in [collection.name for collection in existing_collections]:
    chroma_collection = chroma_client.get_collection(collection_name)
    print(f"Using existing collection '{collection_name}'.")
else:
    chroma_collection = chroma_client.create_collection(collection_name)
    print(f"Created new collection '{collection_name}'.")

# Set up embedding model
embed_model = OllamaEmbedding(
    model_name="snowflake-arctic-embed",
    ollama_additional_kwargs={"prostatic": 0},
)

# Load documents
documents = SimpleDirectoryReader("./data/ravens_web_official_news_10_7_10_14").load_data()

# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(
#     documents, storage_context=storage_context, embed_model=embed_model
# )


# with node
from llama_index.core.node_parser import SentenceSplitter, SimpleNodeParser
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model, transformations=[SimpleNodeParser(chunk_size=512, chunk_overlap=20)]
)

Created new collection 'ravens'.


### Demo

In [ ]:
# Create query engine and perform query
query_engine = index.as_query_engine(similarity_top_k=1)
response = query_engine.query("What's new with Baltimore Ravens special teams?")
# with open("Output.txt", "w") as text_file:
#     text_file.write(str(response))
print(response)

The Ravens' special teams unit has been performing well lately. They've had some clutch plays, including a game-tying field goal by Justin Tucker from 56 yards out against the Bengals. This marked a turning point in the game and showed that Tucker is still one of the most reliable kickers in the league.


# FLARE with ChromaDB

## ChromaDB setup (copy from previous)

In [3]:
# Create Chroma client
chroma_client = chromadb.EphemeralClient()

# Define collection name
collection_name = "ravens"

# Check if the collection already exists
existing_collections = chroma_client.list_collections()

if collection_name in [collection.name for collection in existing_collections]:
    chroma_collection = chroma_client.get_collection(collection_name)
    print(f"Using existing collection '{collection_name}'.")
else:
    chroma_collection = chroma_client.create_collection(collection_name)
    print(f"Created new collection '{collection_name}'.")

# Set up embedding model
embed_model = OllamaEmbedding(
    model_name="snowflake-arctic-embed",
    ollama_additional_kwargs={"prostatic": 0},
)

# Load documents
documents = SimpleDirectoryReader("./data/ravens_web_official_news_10_7_10_14").load_data()

# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(
#     documents, storage_context=storage_context, embed_model=embed_model
# )


# with node
from llama_index.core.node_parser import SentenceSplitter, SimpleNodeParser
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model, transformations=[SimpleNodeParser(chunk_size=512, chunk_overlap=20)]
)

Created new collection 'ravens'.


## FLARE

In [ ]:
# Create query engine and perform query
index_query_engine = index.as_query_engine(similarity_top_k=2)

# create FLARE
from llama_index.core.query_engine import FLAREInstructQueryEngine

flare_query_engine = FLAREInstructQueryEngine(
    query_engine=index_query_engine,
    max_iterations=7,
    verbose=True,
)

## DEMO

In [15]:
response = index_query_engine.query("What's going on with Baltimore Ravens?")

In [16]:
print(response)

The team is having a strong season, as evidenced by their recent division wins. They have also been performing well in game management, which is a key aspect of their success. Additionally, they are showing resilience and determination, as suggested by the comments from one of their players who feels judged despite limited playing time.


In [19]:
flare_response = flare_query_engine.query("What's going on with Baltimore Ravens?")

Query: What's going on with Baltimore Ravens?
Current response: 
Lookahead response: [Search(What is happening with the Baltimore Ravens?)]
Updated lookahead response: Based on the provided information, I will fill in the lookahead templates with the appropriate answers.

**Case 1:**

Previous Response:

Lookahead Template:
Red is for 
Current response:  Based on the provided information, I will fill in the lookahead templates with the appropriate answers.

**Case 1:**

Previous Response:

Lookahead Template:
Red is for
Lookahead response: Based on the existing answer and the lookahead template, here's my attempt to generate the next portion of the answer:

[Search(What color is associated with the Baltimore Ravens?)]


I've used the Search API to look up relevant information about the Baltimore Ravens. If you'd like me to continue generating the answer or if it's complete, please let me know!
Updated lookahead response: Based on the previous response and lookahead template, I will fil

IndexError: list index out of range

In [18]:
print(flare_response)

Based on the provided information, I will fill in the lookahead templates with the appropriate answers.

**First Example**

Previous Response:

Lookahead Template:
Red is for Based on the provided information, I will fill in the lookahead templates with the appropriate answers.

**First Example**

Previous Response:

Lookahead Template:
Red is for Based on the previous response and the query-answer pairs, I will fill in the lookahead template as follows:

Previous Response: The colors on the flag of Ghana have the following meanings. Red is for


# RAG with Qdrant

In [43]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader("./data/ravens_web_official_news_10_7_10_14").load_data()

## Split docs into small chunks

In [44]:
from llama_index.core.node_parser.text import SentenceSplitter
text_parser = SentenceSplitter(
 chunk_size=1024,
)
text_chunks = []
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

## Construct Nodes

In [45]:
from llama_index.core.schema import TextNode
nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
    text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

## Embedding Model

In [46]:
# Set up embedding model
embed_model = OllamaEmbedding(
    model_name="snowflake-arctic-embed",
    ollama_additional_kwargs={"prostatic": 0},
)
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
    node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [52]:
Settings.embed_model = embed_model

## Create Vector Store

In [56]:
import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore
client = qdrant_client.QdrantClient(location=":memory:")
from llama_index.core import (
 VectorStoreIndex,
 StorageContext,
 SimpleDirectoryReader,
)
vector_store = QdrantVectorStore(client=client, collection_name="my_collection")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [57]:
index = VectorStoreIndex.from_documents(
 documents, storage_context=storage_context
)

In [58]:
vector_store.add(nodes)

['82f9b08d-4c2c-4cd6-aa2e-1a139bba5105',
 '301bb0ea-7de1-455a-9283-72bb666047b9',
 '5bfec3d9-a7a2-4fab-82a1-0e811ebcaf15',
 'd2cab947-008a-4bee-9354-d91c65bc99fd',
 '529534f8-aa9f-46f9-9fd8-8b948c0b3090',
 'b6eb8758-2114-4ac4-8e6d-89856ebdf763',
 '60463f07-5294-420a-88f5-431eaef54bdc',
 'cc5b8f71-dc40-4920-80df-311a1e5383ec',
 'ea99810b-4af8-420a-9b8f-721c4a87f612',
 'd8ee931b-e5b8-4581-bf51-4458d7e2bd74',
 'acc2ce09-265e-4b84-b7a7-52a8fbe2ec48',
 'f99e2b81-10de-4136-9138-2830c907a6dc',
 'fe003cdf-9d65-4627-9705-8ba9dd98b9aa',
 '9f6612ff-c101-4ac8-9abb-1e2d4a80ed0e',
 'd68d2757-7cae-4d98-a343-9fbef6bcd9ea',
 '1e4518fb-d3f6-4039-85f6-42b84d9d0ea5',
 '0d67d06a-50d6-4a7a-921e-1bc57247560c',
 '3519df90-e624-44a5-9de5-df11baed480e',
 '86bafcba-38bd-4532-b329-ed103aebb6a6',
 '27b9ce36-0dd6-4727-b404-3c8e12e42b2f',
 '29d4da6d-c263-4c7f-8cec-6dbfcba98591',
 '427b5a35-e798-48b0-bc75-9f2911cee1e7',
 '8eee9fdc-9231-4374-9ab5-89334c5db82d',
 '45675450-355a-47d2-b6b2-dceb5729ce7b',
 'eab98441-d745-

## Query

In [59]:
query_str = "what's new with Baltimore Ravens?"
query_embedding = embed_model.get_query_embedding(query_str)

## Construct Vector Store Query

In [61]:
from llama_index.core.vector_stores import VectorStoreQuery
query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"
vector_store_query = VectorStoreQuery(
 query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

He converted big third down after big third down, did a terrific job re-discovering Mark Andrews, feeding Zay Flowers and finding guys like Tylan Wallace and Charlie Kolar when he needed them. The play to Isaiah Likely in the fourth quarter will run on highlight clips for perpetuity, and it should — but it shouldn't overshadow this gritty, brilliant performance. He looked like a two-time MVP."

The Baltimore Banner's Jonas Shaffer: "One of the first things Derrick Henry said after the game was that Lamar Jackson's the best player in the NFL. After all we saw today — the incredible highs and the very rare lows — who's going to tell him otherwise?"

The Baltimore Banner's Kyle Goon: "A long time from now, when Jackson is enshrined in Canton, they'll play the highlight of him stiff-arming Sam Hubbard to the ground, then throwing a touchdown to Isaiah Likely across his body. The two-time MVP reminded us who he is with a masterpiece in the second half, including three touchdown passes and a

## Parse the Results into a set of nodes

In [62]:
from llama_index.core.schema import NodeWithScore
from typing import Optional
nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
        nodes_with_scores.append(NodeWithScore(node=node, score=score))

## Retriever

In [64]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List

class VectorDBRetriever(BaseRetriever):
    """Retriever over a qdrant vector store."""
    def __init__(
    self,
    vector_store: QdrantVectorStore,
    embed_model: Any,
    query_mode: str = "default",
    similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()
    
    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
        query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
        query_embedding=query_embedding,
        similarity_top_k=self._similarity_top_k,
        mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)
        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
                nodes_with_scores.append(NodeWithScore(node=node, score=score))
        return nodes_with_scores

In [65]:
retriever = VectorDBRetriever(
            vector_store, embed_model, query_mode="default", similarity_top_k=2
            )

## Retriever Query Engine

In [66]:
from llama_index.core.query_engine import RetrieverQueryEngine
query_engine = RetrieverQueryEngine.from_args(
 retriever
)

## Query Results

In [67]:
query_str = "what's new with Baltimore Ravens?"
response = query_engine.query(query_str)
print(str(response))

The Baltimore Ravens have been on a roll lately. Their offense has been particularly impressive, with Lamar Jackson leading the charge. They've shown their ability to adapt and dominate games in different ways, whether it's through the air or on the ground. With players like Zay Flowers emerging as key contributors and Derrick Henry continuing to be a force to be reckoned with, this team is looking fierce.


# Random

In [7]:

from llama_index.core.memory import ChatMemoryBuffer
memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=(
        "You are a chatbot, able to have normal interactions."
    ),
)

response = chat_engine.chat("Is it good for Ravens?")
print(response)

It seems like several things are going well for the Ravens from what you've shared. They had multiple players contribute in the last game, their red-zone offense is doing great with 14 of their last 15 trips resulting in touchdowns, and Justin Tucker broke his skid with a big kick despite tough conditions.


In [ ]:
response = chat_engine.chat("Is it good for Ravens?")
print(response)